# Find Optimal PSF
===========================

- creation 29/06/2016
- author Sylvie Dagoret-Campagne



Find HD158485 with grating

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

from astropy.table import Table
from astropy.table import Column

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [2]:
import photutils
from astropy.stats import sigma_clipped_stats
from photutils import daofind
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

In [3]:
from scipy import stats 
from scipy import ndimage
import os
from datetime import datetime, timedelta

In [4]:
from photutils.background import Background2D

In [5]:
import libMonocamBaseImages           # my tool library written to do that CCD reduction

In [6]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-07-06 14:10:22.343528
fits date format for the analysis :  2016-07-06T14:10:22


## Definitions and Constants

In [7]:
object_name='HD158485_grat'

### input files

In [8]:
path='./HD158485_grat'
rootfilename='AssScImHD158485_grat_' 
#NumStart=1
#NumStop=50

#NumStart=51
#NumStop=100

NumStart=1
NumStop=13

### output file (table)

In [9]:
outputtablefile='HD158485_grat_1-13_TablePSF.fits'

### make the filelist

In [10]:
filelist=libMonocamBaseImages.BuildFilelist(path,rootfilename,start=NumStart,stop=NumStop,nbchar=1)

In [11]:
filelist

['./HD158485_grat/AssScImHD158485_grat_1.fits',
 './HD158485_grat/AssScImHD158485_grat_2.fits',
 './HD158485_grat/AssScImHD158485_grat_3.fits',
 './HD158485_grat/AssScImHD158485_grat_4.fits',
 './HD158485_grat/AssScImHD158485_grat_5.fits',
 './HD158485_grat/AssScImHD158485_grat_6.fits',
 './HD158485_grat/AssScImHD158485_grat_7.fits',
 './HD158485_grat/AssScImHD158485_grat_8.fits',
 './HD158485_grat/AssScImHD158485_grat_9.fits',
 './HD158485_grat/AssScImHD158485_grat_10.fits',
 './HD158485_grat/AssScImHD158485_grat_11.fits',
 './HD158485_grat/AssScImHD158485_grat_12.fits',
 './HD158485_grat/AssScImHD158485_grat_13.fits']

## Read Input files

In [12]:
allchannelallsciimages = []  # list of 16 lists of images series 
exposures_list = []        # sequential list of the exposures of the sky flats 
header_list = []           # list of headers
data_list = []
time_list = []                # date and time
basefile_list = []         # basefilename
dateobs_list = [] 
# get the primary block headers:
for image_file in filelist: 
    print image_file
    hdu_list = fits.open(image_file)
    basefile_list.append(os.path.basename(image_file))
    header=hdu_list[0].header
    exposure=header['EXPOSURE']
    exposures_list.append(exposure)
    dateobs_list.append(header['DATE-OBS'])
    header_list.append(header)
    data=ccdproc.CCDData.read(image_file, hdu=0,unit='adu') 
    data_list.append(data)

./HD158485_grat/AssScImHD158485_grat_1.fits
./HD158485_grat/AssScImHD158485_grat_2.fits
./HD158485_grat/AssScImHD158485_grat_3.fits
./HD158485_grat/AssScImHD158485_grat_4.fits
./HD158485_grat/AssScImHD158485_grat_5.fits
./HD158485_grat/AssScImHD158485_grat_6.fits
./HD158485_grat/AssScImHD158485_grat_7.fits
./HD158485_grat/AssScImHD158485_grat_8.fits
./HD158485_grat/AssScImHD158485_grat_9.fits
./HD158485_grat/AssScImHD158485_grat_10.fits
./HD158485_grat/AssScImHD158485_grat_11.fits
./HD158485_grat/AssScImHD158485_grat_12.fits
./HD158485_grat/AssScImHD158485_grat_13.fits


In [13]:
#basefile_list

## For control

uncomment for control

In [14]:
index=0

In [15]:
#print exposures_list[index]

In [16]:
#header_list[index]

In [17]:
#plt.imshow(data_list[index])

In [18]:
#bkg= Background2D(data_list[index], (100, 100), filter_size=(3, 3),method='median')

In [19]:
#data_list[index].data-bkg.background

## Background subtraction

In [20]:
correctedimage_list = []

In [21]:
for data in data_list:
    bkg= Background2D(data, (100, 100), filter_size=(3, 3),method='median')
    newimage=data-bkg.background
    correctedimage_list.append(newimage)

## Calculation of PSF

In [22]:
NBIMAGES=len(correctedimage_list)

In [23]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (average, np.sqrt(variance))

In [24]:
image_psf=np.zeros((NBIMAGES,6))

In [25]:
DELTA_NBINSX=10
DELTA_NBINSY=10

In [26]:
# loop on images
index=-1
for image in correctedimage_list:
    index=index+1
    mean, median, std = sigma_clipped_stats(image, sigma=10.0, iters=5) 
    sources = daofind(image - median, fwhm=3.0, threshold=50.*std)
    selected_stars=sources.as_array()
    NBSTARS=selected_stars.shape[0]
    print 'image {} ==> NBSTARS = {}'.format(index,NBSTARS)
    image_psf[index,0]=index
    image_psf[index,1]=NBSTARS
    star_psfx=np.zeros(NBSTARS)
    star_psfy=np.zeros(NBSTARS)
    # loop on stars
    for istar in range(NBSTARS):
        X = int(selected_stars[istar][1])
        Y = int(selected_stars[istar][2])
        prf_image = image[Y-DELTA_NBINSY:Y+DELTA_NBINSY,X-DELTA_NBINSX:X+DELTA_NBINSX]
        oneprfX=prf_image.sum(axis=0)
        oneprfY=prf_image.sum(axis=1)
        if oneprfX.sum() == 0 or oneprfY.sum() == 0:
            star_psfx[istar]=0
            star_psfy[istar]=0
        else:
            posX,sigX=weighted_avg_and_std(np.arange(oneprfX.shape[0]),oneprfX)
            posY,sigY=weighted_avg_and_std(np.arange(oneprfY.shape[0]),oneprfY)
            star_psfx[istar]=sigX
            star_psfy[istar]=sigY
        
    all_sigx=star_psfx[np.logical_not(np.isnan(star_psfx))]
    all_sigy=star_psfy[np.logical_not(np.isnan(star_psfy))]
    all_sigx=all_sigx[all_sigx>2.4]
    all_sigy=all_sigy[all_sigy>2.4]
    print 'average prf(x) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigx),all_sigx.std(),np.median(all_sigx)*0.4*2.36,all_sigx.std()*0.4*2.26)
    print 'average prf(y) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigy),all_sigy.std(),np.median(all_sigy)*0.4*2.36,all_sigy.std()*0.4*2.26)
    image_psf[index,2]=np.median(all_sigx)
    image_psf[index,3]=np.median(all_sigy)
    image_psf[index,4]=all_sigx.std()
    image_psf[index,5]=all_sigy.std()

image 0 ==> NBSTARS = 32
average prf(x) = 4.60 +/- 0.67 pixels ==> psf  4.34 +/- 0.61 arcsec 
average prf(y) = 4.78 +/- 0.63 pixels ==> psf  4.51 +/- 0.57 arcsec 


/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


image 1 ==> NBSTARS = 20
average prf(x) = 4.37 +/- 0.25 pixels ==> psf  4.12 +/- 0.23 arcsec 
average prf(y) = 4.28 +/- 0.48 pixels ==> psf  4.04 +/- 0.43 arcsec 
image 2 ==> NBSTARS = 21
average prf(x) = 3.12 +/- 0.14 pixels ==> psf  2.94 +/- 0.13 arcsec 
average prf(y) = 3.38 +/- 0.29 pixels ==> psf  3.19 +/- 0.26 arcsec 
image 3 ==> NBSTARS = 44
average prf(x) = 3.20 +/- 0.25 pixels ==> psf  3.02 +/- 0.23 arcsec 
average prf(y) = 3.54 +/- 0.73 pixels ==> psf  3.34 +/- 0.66 arcsec 
image 4 ==> NBSTARS = 40
average prf(x) = 3.33 +/- 0.20 pixels ==> psf  3.15 +/- 0.18 arcsec 
average prf(y) = 3.70 +/- 0.71 pixels ==> psf  3.49 +/- 0.64 arcsec 
image 5 ==> NBSTARS = 51
average prf(x) = 3.22 +/- 0.23 pixels ==> psf  3.04 +/- 0.21 arcsec 
average prf(y) = 3.58 +/- 0.78 pixels ==> psf  3.38 +/- 0.71 arcsec 
image 6 ==> NBSTARS = 43
average prf(x) = 3.06 +/- 0.27 pixels ==> psf  2.89 +/- 0.25 arcsec 
average prf(y) = 3.55 +/- 0.79 pixels ==> psf  3.35 +/- 0.72 arcsec 
image 7 ==> NBSTARS = 

## Create astropy table

In [27]:
t=Table(rows=image_psf,names=('num','nbstars','prfx','pfry','sig_prfx','sig_prfy'),dtype=('i4','i4','f8','f8','f8','f8'))

In [28]:
t

num,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,int32,float64,float64,float64,float64
0,32,4.59516684466,4.77633085449,0.669748518204,0.633033961784
1,20,4.36764093337,4.27710462183,0.253070362051,0.480219677255
2,21,3.11745978946,3.37662120697,0.141785922328,0.291356294233
3,44,3.20254070405,3.54157179463,0.253742310559,0.732325225588
4,40,3.3337821852,3.69825493689,0.197919263153,0.712456743168
5,51,3.22342719729,3.57597468963,0.22727713948,0.784218480413
6,43,3.06221826786,3.54834731387,0.274848952559,0.794321902288
7,50,3.20182477857,3.46410505349,0.518441965741,0.916487419153
8,7,3.12579168451,3.48152512138,0.106598705227,0.0555052401655


In [29]:
expo = Column(exposures_list, name='exposure')
file = Column(basefile_list, name='file')
time = Column(dateobs_list,name='time')

In [30]:
t.add_column(expo, index=1)
t.add_column(time, index=1)
t.add_column(file, index=1)

In [31]:
t

num,file,time,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,str28,str23,float64,int32,float64,float64,float64,float64
0,AssScImHD158485_grat_1.fits,2016-05-11T05:58:59.852,1.5,32,4.59516684466,4.77633085449,0.669748518204,0.633033961784
1,AssScImHD158485_grat_2.fits,2016-05-11T05:59:35.430,1.5,20,4.36764093337,4.27710462183,0.253070362051,0.480219677255
2,AssScImHD158485_grat_3.fits,2016-05-11T06:00:39.034,1.5,21,3.11745978946,3.37662120697,0.141785922328,0.291356294233
3,AssScImHD158485_grat_4.fits,2016-05-11T06:04:28.397,5.0,44,3.20254070405,3.54157179463,0.253742310559,0.732325225588
4,AssScImHD158485_grat_5.fits,2016-05-11T06:04:37.465,5.0,40,3.3337821852,3.69825493689,0.197919263153,0.712456743168
5,AssScImHD158485_grat_6.fits,2016-05-11T06:04:46.678,5.0,51,3.22342719729,3.57597468963,0.22727713948,0.784218480413
6,AssScImHD158485_grat_7.fits,2016-05-11T06:04:55.971,5.0,43,3.06221826786,3.54834731387,0.274848952559,0.794321902288
7,AssScImHD158485_grat_8.fits,2016-05-11T06:05:04.595,5.0,50,3.20182477857,3.46410505349,0.518441965741,0.916487419153
8,AssScImHD158485_grat_9.fits,2016-05-11T06:05:58.173,0.5,7,3.12579168451,3.48152512138,0.106598705227,0.0555052401655


## Write output file

In [32]:
t.write(outputtablefile,format='fits')

## Pandas

In [33]:
df=t.to_pandas()

In [34]:
df.describe()

,num,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
count,13.00000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,6.00000,2.461538,26.307692,3.392544,3.677319,0.285885,0.544807
std,3.89444,2.125848,17.589697,0.502267,0.409934,0.160303,0.259578
min,0.00000,0.500000,5.000000,2.919461,3.276756,0.106599,0.055505
25%,3.00000,0.500000,10.000000,3.125792,3.464105,0.170638,0.374817
50%,6.00000,1.500000,21.000000,3.202541,3.548347,0.253070,0.633034
75%,9.00000,5.000000,43.000000,3.371829,3.698255,0.344703,0.732325
max,12.00000,5.000000,51.000000,4.595167,4.776331,0.669749,0.916487
